In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors

### TODO: 1. Functions of MC1

In [ ]:
# TODO: Calculate the FDV
def divalent_nodes_fraction(smiles):
    mol=Chem.MolFromSmiles(smiles)
    atom_number = 0
    divalent_node = 0
    
    for atom in mol.GetAtoms():
        atom_number += 1
        degree = atom.GetDegree()

        if degree == 2:
            divalent_node += 1 
        else:
            continue

    divalent_ratio = round(divalent_node/atom_number,2)
    return divalent_ratio

In [ ]:
# TODO: MC1 calculation
def mc1(smiles):

    mc1 = 1- divalent_nodes_fraction(smiles)
    
    return mc1

### TODO: 2. Functions of MC2

In [ ]:
# TODO: MC2 calculation - Count non-divalent atoms that are not in the C=O-X double bonds 
def mc2(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError("Invalid SMILES string.")

    atoms_in_C_O_X_double_bond = set()

    for bond in mol.GetBonds():
        # Check for a C=O double bond
        if bond.GetBondType() == Chem.rdchem.BondType.DOUBLE:
            begin_atom = bond.GetBeginAtom()
            end_atom = bond.GetEndAtom()

            if {begin_atom.GetAtomicNum(), end_atom.GetAtomicNum()} == {6, 8}:  # C and O
                # Identify which one is the carbon
                carbon = begin_atom if begin_atom.GetAtomicNum() == 6 else end_atom
                oxygen = end_atom if carbon == begin_atom else begin_atom

                # Check carbon's neighbors for N or O (excluding the double-bonded O)
                for neighbor in carbon.GetNeighbors():
                    if neighbor.GetIdx() != oxygen.GetIdx() and neighbor.GetAtomicNum() in [7, 8]:
                        atoms_in_C_O_X_double_bond.update([carbon.GetIdx(), oxygen.GetIdx()])
                        break  # Only need one N/O neighbor to satisfy the condition

    # Count non-divalent atoms not in C=O-X double bonds
    count = 0
    for atom in mol.GetAtoms():
        if atom.GetDegree() != 2 and atom.GetIdx() not in atoms_in_C_O_X_double_bond:
            count += 1

    return count


### TODO: 3. Calculation of MC1 and MC2

In [ ]:
mc1("COC1=C(O)C=C(CC(=O)O)C=C1Br")

In [ ]:
mc2("COC1=C(O)C=C(CC(=O)O)C=C1Br")